Задание:

1. Задетектить всех людей на видео.
2. Задетектить их лица (+ эмоции).
3. Трекинг перемещения.
4. Через лендмарки - положение человека (упал, сел, ...) и темп передвижения.
5. Отметить подозрительных персон.

План:

1. Использовать YOLO для детекции людей в кадре (построим bounding box для каждого человека; позволяет обрабатывать несколько людей одновременно; позволит нам передавать в MediaPipe только нужные области (crops), а не весь кадр). - отвечает за поиск людей в кадре.
2. MediaPipe получит box-ы с людьми, которых задетектила YOLO, и будет извлекать landmarks (Pose (33 ключевые точки тела), Face Mesh (468 точек лица), Hands (21 точка руки)). - отвечает за позу и эмоции человека.

In [100]:
import cv2
from ultralytics import YOLO
import mediapipe as mp

In [101]:
import math
import numpy as np

In [102]:
# Модель YOLO для детекции людей
yolo_model = YOLO("yolov8s.pt")

In [103]:
from google.colab.patches import cv2_imshow

In [104]:
# MediaPipe для позы и лица
mp_pose = mp.solutions.pose.Pose() # модуль для анализа позы (скелет)
mp_face = mp.solutions.face_mesh.FaceMesh() # сеточка из точек для анализа лица
mp_draw = mp.solutions.drawing_utils # модуль для рисования точек и соединений на изображении

In [66]:
from google.colab import files

uploaded = files.upload()

Saving people16.mp4 to people16.mp4


In [105]:
# Входное видео
cap = cv2.VideoCapture("people16.mp4")

In [106]:
# Параметры выходного видео
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # кодек mp4
fps = int(cap.get(cv2.CAP_PROP_FPS))      # частота кадров исходного видео
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [107]:
# Выходное видео
out = cv2.VideoWriter("/content/people16_result.mp4", fourcc, fps, (width, height))

In [108]:
prev_centers = {}

In [109]:
# Порог уверенности, чтобы лишнего в MediaPipe не передавать
confidence_threshold = 0.5

In [110]:
import random
# генерируем разные цвета для разных людей
# словарь для хранения цветов по ID
id_colors = {}

def get_color(track_id):
    if track_id not in id_colors:
        # генерируем случайный цвет
        id_colors[track_id] = (
            random.randint(50, 255),
            random.randint(50, 255),
            random.randint(50, 255)
        )
    return id_colors[track_id]

In [111]:
# цикл обработки кадров
for result in yolo_model.track(source="people.mp4", tracker="bytetrack.yaml", stream=True):
    frame = result.orig_img

    for box in result.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        if cls_id != 0 or conf < confidence_threshold: # нас интересуют только люди
            continue

        x1, y1, x2, y2 = map(int, box.xyxy[0])
        track_id = int(box.id[0]) if box.id is not None else -1 # закрепляем track_id за человеком

        roi = frame[y1:y2, x1:x2]
        roi_h, roi_w = roi.shape[:2]

        color = get_color(track_id)  # цвет для этого ID

        # рисуем рамку вокруг человека
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        # анализируем позу и отрисовываем
        pose_results = mp_pose.process(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
        if pose_results.pose_landmarks:
            # переводим лендмарки в координаты изначального фрейма, а не нормализованные по кропу
            lm = pose_results.pose_landmarks.landmark
            lx_px = lm[mp.solutions.pose.PoseLandmark.LEFT_HIP].x * roi_w + x1
            ly_px = lm[mp.solutions.pose.PoseLandmark.LEFT_HIP].y * roi_h + y1
            rx_px = lm[mp.solutions.pose.PoseLandmark.RIGHT_HIP].x * roi_w + x1
            ry_px = lm[mp.solutions.pose.PoseLandmark.RIGHT_HIP].y * roi_h + y1
            cx, cy = (lx_px + rx_px) / 2, (ly_px + ry_px) / 2

            if track_id in prev_centers:
                dx = cx - prev_centers[track_id][0]
                dy = cy - prev_centers[track_id][1]
                dist = math.sqrt(dx**2 + dy**2)
                speed = dist * fps

                cv2.putText(frame, f"ID {track_id} Speed: {speed:.1f} px/s",
                            (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

            prev_centers[track_id] = (cx, cy)

            roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
            mp_draw.draw_landmarks(roi_rgb, pose_results.pose_landmarks,
                                   mp.solutions.pose.POSE_CONNECTIONS,
                                   mp_draw.DrawingSpec(color=color, thickness=2, circle_radius=2),
                                   mp_draw.DrawingSpec(color=color, thickness=2))
            roi[:] = cv2.cvtColor(roi_rgb, cv2.COLOR_RGB2BGR)

        # анализируем лицо и отрисовываем
        face_results = mp_face.process(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
        if face_results.multi_face_landmarks:
            roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
            for landmarks in face_results.multi_face_landmarks:
                mp_draw.draw_landmarks(roi_rgb, landmarks,
                                       mp.solutions.face_mesh.FACEMESH_TESSELATION,
                                       mp_draw.DrawingSpec(color=color, thickness=1, circle_radius=1),
                                       mp_draw.DrawingSpec(color=color, thickness=1))
            roi[:] = cv2.cvtColor(roi_rgb, cv2.COLOR_RGB2BGR)

    out.write(frame)


video 1/1 (frame 1/1302) /content/people.mp4: 320x640 8 persons, 1 bus, 11.3ms
video 1/1 (frame 2/1302) /content/people.mp4: 320x640 8 persons, 1 bus, 10.7ms
video 1/1 (frame 3/1302) /content/people.mp4: 320x640 8 persons, 1 motorcycle, 1 bus, 10.7ms
video 1/1 (frame 4/1302) /content/people.mp4: 320x640 8 persons, 1 motorcycle, 1 bus, 10.7ms
video 1/1 (frame 5/1302) /content/people.mp4: 320x640 8 persons, 1 motorcycle, 1 train, 10.7ms
video 1/1 (frame 6/1302) /content/people.mp4: 320x640 8 persons, 1 motorcycle, 10.7ms
video 1/1 (frame 7/1302) /content/people.mp4: 320x640 9 persons, 1 motorcycle, 11.7ms
video 1/1 (frame 8/1302) /content/people.mp4: 320x640 9 persons, 1 motorcycle, 15.5ms
video 1/1 (frame 9/1302) /content/people.mp4: 320x640 9 persons, 1 motorcycle, 11.2ms
video 1/1 (frame 10/1302) /content/people.mp4: 320x640 9 persons, 1 train, 10.7ms
video 1/1 (frame 11/1302) /content/people.mp4: 320x640 10 persons, 1 motorcycle, 1 train, 10.8ms
video 1/1 (frame 12/1302) /content/pe

In [112]:
cap.release()
out.release()
cv2.destroyAllWindows()
"Видео сохранено: people_output.mp4"

'Видео сохранено: people_output.mp4'

Вывод о проделанной работе: YOLO хорошо справляется с детекцией людей, даже если их много, они далеко, со спины и т.д. С отрисовкой позы и нанесением лендмарок на лицо от mediapipe тоже нет проблем. Проблема есть с трекингом. При перекрытии, исчезании из кадра, человеку потом присваивается новый ID. Кроме того, что это априори не очень, так ещё и алгоритм расчёта скорости сбивается, так как предыдущие данные исчезают. И я только в конце поняла, что мой текущий алгоритм расчёта скорости работает хорошо, если камера статична, а в моём видео, камера движется и получается, что люди, которые сидят или неподвижно стоят, тоже имеют нормальную такую скорость, так как положение во фрейме меняется из-за перемещения камеры относительно них, надо как-то подумать относительно чего считать, если всё движется.